This notebook contains a visual representation of the steps to create the tables for our Fantasy Football Database using existing datasets.

Dataset: https://www.kaggle.com/zynicide/nfl-football-player-stats

In [1]:
import pandas as pd
import json
import pickle

## Read in dataset

**Create profiles table**

In [2]:
with open(r'../Data/Datasets/profiles.json', 'r') as f:
    data = json.load(f)

In [3]:
data[0]

{'player_id': 1809,
 'name': 'Robert Blackmon',
 'position': 'DB',
 'height': '6-0',
 'weight': '208',
 'current_team': None,
 'birth_date': '1967-05-12',
 'birth_place': 'Bay City, TX',
 'death_date': None,
 'college': 'Baylor',
 'high_school': 'Van Vleck, TX',
 'draft_team': 'Seattle Seahawks',
 'draft_round': '2',
 'draft_position': '34',
 'draft_year': '1990',
 'current_salary': None,
 'hof_induction_year': None}

In [4]:
cleaned_data = []
for entry in data:
    cleaned_data.append({'playerID': entry['player_id'], 'name': entry['name'].strip(' '), 'position': entry['position'].split('-')[0],
                        'team': entry['current_team']})

In [5]:
profile = pd.DataFrame(cleaned_data)
profile.set_index('playerID', inplace=True)
profile.head()

,name,position,team
playerID,,,
1809,Robert Blackmon,DB,None
23586,Dean Wells,LB,None
355,Kiko Alonso,ILB,Miami Dolphins
18182,Steve Ramsey,QB,None
16250,Cory Nelms,CB,None


**Read in Games Data**

In [6]:
with open(r'../Data/Datasets/games.json', 'r') as f:
    games = json.load(f)

In [7]:
gameframe = pd.DataFrame(games)
gameframe = gameframe[gameframe['year'] == '2016']  # Limit to 2016 year
gameframe.head()

,player_id,year,date,game_number,age,team,game_location,opponent,game_won,player_team_score,...,defense_interception_yards,defense_interception_touchdowns,defense_safeties,point_after_attemps,point_after_makes,field_goal_attempts,field_goal_makes,punting_attempts,punting_yards,punting_blocked
290,355,2016,2016-09-11,1,26-028,MIA,A,SEA,False,10,...,0,0,0,0,0,0,0,0,0,0
291,355,2016,2016-09-18,2,26-035,MIA,A,NWE,False,24,...,0,0,0,0,0,0,0,0,0,0
292,355,2016,2016-09-25,3,26-042,MIA,H,CLE,True,30,...,0,0,0,0,0,0,0,0,0,0
293,355,2016,2016-09-29,4,26-046,MIA,A,CIN,False,7,...,0,0,0,0,0,0,0,0,0,0
294,355,2016,2016-10-09,5,26-056,MIA,H,TEN,False,17,...,0,0,0,0,0,0,0,0,0,0


## Create Initial Offense and Defense Tables

**Create Offense Table**

In [12]:
offense = gameframe[['player_id', 'game_number', 'passing_yards', 'rushing_yards', 'receiving_yards', 
                          'passing_touchdowns', 'receiving_touchdowns', 'rushing_touchdowns', 'passing_interceptions']]

offense = offense.rename(columns={'player_id': 'playerID', 'game_number': 'gameNumber', 'passing_yards': 'passYards', 
                                  'rushing_yards': 'rushYards', 'receiving_yards': 'recYards', 
                                  'passing_interceptions': 'interceptions'})

offense['touchdowns'] = offense['passing_touchdowns'] + offense['receiving_touchdowns'] + offense['rushing_touchdowns']
offense.drop(['passing_touchdowns', 'receiving_touchdowns', 'rushing_touchdowns'], axis=1, inplace=True)

In [13]:
offense.head()

,playerID,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns
290,355,1,0,0,0,0,0
291,355,2,0,0,0,0,0
292,355,3,0,0,0,0,0
293,355,4,0,0,0,0,0
294,355,5,0,0,0,0,0


In [14]:
offense['gameNumber'] = offense['gameNumber'].astype(int)
offense = offense[offense['gameNumber'] < 17]
offense.head()

,playerID,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns
290,355,1,0,0,0,0,0
291,355,2,0,0,0,0,0
292,355,3,0,0,0,0,0
293,355,4,0,0,0,0,0
294,355,5,0,0,0,0,0


In [15]:
offensive_profiles = profile[(profile['position'] == 'QB') | 
                             (profile['position'] == 'WR') | 
                             (profile['position'] == 'RB') | 
                             (profile['position'] == 'TE')]

offense = pd.merge(offense, offensive_profiles, how='inner', on='playerID')
offense.dropna(axis=0, inplace=True)
offense.drop(['playerID', 'team'], axis=1, inplace=True)

offense = offense.rename(columns={'name': 'playerID'})
offense.head()

,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns,playerID,position
4,1,0,0,49,0,0,Emmanuel Sanders,WR
5,2,0,0,39,0,0,Emmanuel Sanders,WR
6,3,0,0,117,0,2,Emmanuel Sanders,WR
7,4,0,0,88,0,1,Emmanuel Sanders,WR
8,5,0,0,80,0,0,Emmanuel Sanders,WR


In [16]:
with open(r'../Data/Pickles/offense.pickle', 'wb') as f:
    pickle.dump(offense, f)

**Create Defense Table**

In [17]:
defense = gameframe[['player_id', 'game_number', 'defense_sacks', 'defense_interceptions', 'defense_interception_touchdowns',
                    'defense_safeties', 'punting_blocked']]

defense = defense.rename(columns={'player_id': 'playerID', 'game_number': 'gameNumber', 'defense_interceptions': 'interceptions',
                                 'defense_interception_touchdowns': 'touchdowns', 'defense_safeties': 'safeties',
                                 'punting_blocked': 'blocks', 'defense_sacks': 'sacks'})

defense['gameNumber'] = defense['gameNumber'].astype(int)
defense = defense[defense['gameNumber'] < 17]

defense_profiles = profile[(profile['position'] != 'QB') & 
                           (profile['position'] != 'WR') & 
                           (profile['position'] != 'RB') & 
                           (profile['position'] != 'TE')]

defense = pd.merge(defense, defense_profiles, how='inner', on='playerID')
defense.dropna(axis=0, inplace=True)
defense = defense.groupby(['team', 'gameNumber'], as_index=False)[['sacks', 'interceptions', 'touchdowns', 'safeties', 'blocks']].sum()
defense = defense.rename(columns={'team': 'playerID'})

In [18]:
defense.head()

,playerID,gameNumber,sacks,interceptions,touchdowns,safeties,blocks
0,Arizona Cardinals,1,2.0,0,0,0,0
1,Arizona Cardinals,2,2.0,2,0,0,0
2,Arizona Cardinals,3,2.0,1,0,1,0
3,Arizona Cardinals,4,2.0,0,0,0,0
4,Arizona Cardinals,5,2.0,0,0,0,0


In [19]:
with open(r'../Data/Pickles/defense.pickle', 'wb') as f:
    pickle.dump(defense, f)

## Write to CSV

In [20]:
defense.to_csv(r'../Data/CSV/defense.csv', header=False, index=False)

In [21]:
offense.to_csv(r'../Data/CSV/offense.csv', header=False, index=False)